In [69]:
import requests

# Create Pairings Record

In [70]:
# id to be auto generated
division = 981121
tournament = 26620
url = "https://www.tabroom.com/index/tourn/postings/round.mhtml?tourn_id=26620&round_id=981121"
# url = "https://www.tabroom.com/index/tourn/postings/round.mhtml?tourn_id=29541&round_id=1094146"

In [71]:
# MAJOR SLOWDOWN: parse_rounds calls same url as this function

from bs4 import BeautifulSoup


# response = requests.get('https://www.tabroom.com/index/tourn/results/round_results.mhtml?'+url.split("?")[-1])
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
first_table = soup.find('table')



In [72]:
import pandas as pd


table_data = []
headers = []

# Process table headers if they exist
thead = first_table.find('thead')
if thead:
	header_row = thead.find('tr')
	if header_row:
		headers = [th.get_text(strip=True) for th in header_row.find_all('th')]

# Process table rows
tbody = first_table.find('tbody')
if tbody:
	rows = tbody.find_all('tr')
	for row in rows:
		cols = row.find_all('td')
		row_data = []
		for col in cols:
			# Get text content
			text = col.get_text(strip=True)
			# Find all links in the cell
			links = [a['href'] for a in col.find_all('a', href=True)]
			# Append text and links as a tuple
			row_data.append((text, links))
		table_data.append(row_data)
df = pd.DataFrame(table_data, columns=headers)

In [73]:
# Create a boolean mask of columns where the first row's second element is not an empty list
columns_with_links = [idx for idx in range(len(df.columns)) if df.iloc[0, idx][1] != []]

# Create a new DataFrame that only includes the filtered columns
df_filtered = df.iloc[:, columns_with_links].copy()

# Display the filtered DataFrame
# print(df_filtered)
df_filtered


,,,Judge
0,"(Redmond Neha Mahesh & Nikki Taleghani, [/inde...","(Overlake Ammar Ghouse & Caitrin Walker, [/ind...","(Hans Soderquist, [judge.mhtml?judge_id=197430..."
1,(Henrietta Lacks Health and Bioscience London ...,"(Ferris Ben Elrod & John Garten, [/index/tourn...","(Girija Chintalapati, [judge.mhtml?judge_id=19..."
2,(Gonzaga Prep William Kladar & Max Johnson & S...,"(Tahoma Rex Dugan & Hailey Sato, [/index/tourn...","(Emmett Simmons, [judge.mhtml?judge_id=1974939..."
3,"(Interlake Clarence Guo & Daniel Jung, [/index...","(Overlake Musab Chummun & Erik Vank, [/index/t...","(Chris Price, [judge.mhtml?judge_id=1966977&to..."
4,"(Mt. Spokane Ellie Henshaw & Beau Tampien, [/i...","(Union Elliot Holden & Alice Schaak, [/index/t...","(Paul Lorimer, [judge.mhtml?judge_id=1979687&t..."
5,"(Eastside Prep Zubin Oommen & Jack Little, [/i...",(Mountlake Terrace Nathaniel Ballard & Seth Me...,"(Amy Chapman, [judge.mhtml?judge_id=1985314&to..."
6,"(Mount Vernon Dawn Rosas & Ailene Valles, [/in...",(Battle Ground Elizabeth McAleer & Clarity Coo...,"(Skye Cordero, [judge.mhtml?judge_id=1971595&t..."
7,"(Ferris Anvi Sehgal & Madeleine Schreiber, [/i...","(Gig Harbor Logan Thoms & Madison Clement, [/i...","(Thomas Cline, [judge.mhtml?judge_id=1985781&t..."
8,(Edmonds-Woodway Kaylee Meyers & Audrey Gibson...,"(Glacier Peak Naomi Varghese & Maisha Shirazi,...","(Kaveh Dilmaghani, [judge.mhtml?judge_id=19669..."
9,"(Kamiakin Ryan Kirby & Parker Vladimiroff, [/i...",(Bainbridge Evelyn Cantwell & Eilise O'Hartiga...,"(Ying Ning, [judge.mhtml?judge_id=1985315&tour..."


In [74]:
df_filtered.shape


(16, 3)

In [75]:
for i in list(df_filtered.iterrows())[:1]:
	print(i[1][1])

('Overlake Ammar Ghouse & Caitrin Walker', ['/index/tourn/postings/entry_record.mhtml?tourn_id=26620&entry_id=4746833'])


/var/folders/6w/yqthf3cx1v1gtm_3th5g0nk00000gp/T/ipykernel_69748/3617904887.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(i[1][1])


In [129]:
def create_judge(given_role, url, content, pairing_id, first_url):
	print(url)
	response = requests.get('https://www.tabroom.com/index/tourn/postings/'+url)
	# soup = BeautifulSoup(response.text, 'html.parser')
	# first_table = soup.find('table')
	# print(f"Judge: {content}")

def create_team(given_role, url, content, pairing_id):
	print(f"Team: {content}")

In [130]:
def create_team_or_judge(given_role, content, pairing_id, first_url):
	url = content[1][0]
	if url.split(".")[0]=='judge':
		create_judge(given_role, url, content, pairing_id, first_url)
	else:
		create_team(given_role, url, content, pairing_id)

# TODO make this function spawn a bunch of concurent processes or better yet make a work pool
for match in list(df_filtered.itertuples())[1:2]:
	if 'judge' not in str(match):
		print(match, " is a bye")
		# return
	first_url = match[1][1][0]
	# TODO check if there is an exisitng pairing with the same division, tournament, and first ulr, if not,
	# TODO make pairing row and pass its id to create team or judge
	pairing_id = 1
	for index, content in enumerate(match[1:]):
		role = df_filtered.columns[index]
		if role == '': role = 'Competitor'
		create_team_or_judge(role, content, pairing_id, first_url)


Team: ('Henrietta Lacks Health and Bioscience London Lockwood & Kenneth', ['/index/tourn/postings/entry_record.mhtml?tourn_id=26620&entry_id=4778561'])
Team: ('Ferris Ben Elrod & John Garten', ['/index/tourn/postings/entry_record.mhtml?tourn_id=26620&entry_id=4756380'])
judge.mhtml?judge_id=1985922&tourn_id=26620


# Create Judges Record

# Create Teams Record